# Task 1 a) Process Quadrat in MP4 Video
This notebook is configured to read a video specified by `file`, then extract the lines that compose the quadrat and estimate the appropriate corner points for cropping its 
contents. The output is a lower resolution MP4 video that is annotated with the found lines 
and corner points.

In [5]:
# Check if notebook is running in Colab or local workstation
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Search for all video files on Google Drive...
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_PATH = r'/content/drive/My Drive/Data'
    
    # cd into git repo so python can find utils
    %cd '/content/drive/My Drive/cciw-zebra-mussel'
    
    # clone repo, install packages
else:
    DATA_PATH = r'/media/angus/cciw/Data'

In [6]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from utils import draw_lines

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
#%load_ext autoreload
#%autoreload 2

figsize = (8, 6)
save_figures = False

In [7]:
file = 'GLNI_12-1_2016-07-11_video-1.mp4'
#file = 'GLNI_456-3_2015-07-17_video-1.mp4'

In [8]:
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))
videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')

video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok
video_path = os.path.join(video_path, file)
print('Loading video: ', video_path)

Loading video:  /media/angus/cciw/Data/Videos_and_stills/GLNI/12/2016/Jul.11/Videos/Quad1/GLNI_12-1_2016-07-11_video-1.mp4


In [9]:
"""These are meta-parameters of the Probabilistic Hough Line Transform, 
note there are additional params in cell 6 which we set according to 
the input resolution.

@param rho Distance resolution of the accumulator (pixels)"""
rho = 1  

"""
@param theta Angle resolution of the accumulator (radians)

Suggest to use a value no less than np.pi/90 else too many 
spurious lines will be found. theta=np.pi/45 means lines 
must differ by 4 degrees to be considered distinct."""
theta = np.pi / 45

In [10]:
cap = cv2.VideoCapture(video_path)
sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Raw input resolution', sz)

Raw input resolution (1920, 1080)


In [11]:
# read first frame to adjust resolution of output stream
ret, im = cap.read() 

# set additional meta-parameters according to input res
if sz[0] == 1440:
    """x_trim and y_trim are used to remove black padding 
    which triggers spurious edges"""
    x_trim, y_trim = 1, 145
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    crop_frame_border = True
    
    """@param canny_thresh# hysteresis values for Canny edge 
    detector, input to HoughLines"""
    canny_thresh1, canny_thresh2 = 60, 300
    
    """@param threshold Accumulator threshold, return 
    lines with more than threshold of votes. (intersection points)"""
    threshold = 125
    
    """@param minLineLength Minimum line length. 
    Line segments shorter than that are rejected. (pixels)"""
    mLL = 400
    
    """@param maxLineGap Maximum allowed gap between points 
    on the same line to link them. (pixels)"""
    mLG = 150
else:
    # params as described above
    canny_thresh1, canny_thresh2 = 30, 300
    threshold = 125
    mLG, mLL = 250, 600
    crop_frame_border = False

"""this method may downsample, so set the video writer 
resolution to the processed image resolution"""
img, edges = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1)    
sz = (img.shape[1], img.shape[0])
print(sz)

(1440, 810)


In [12]:
if cap.isOpened():
    fps = 20
    vout = cv2.VideoWriter()
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vout.open(file.split('.')[0] + '-quadrat-demo.mp4', fourcc, fps, sz, True)
    print('Opened stream for writing, output resolution is', sz)
else:
    print('cap is not open')

Opened stream for writing, output resolution is (1440, 810)


In [13]:
currentFrame = 0

"""it can take 30s-1min to process entire video, 
can optionally process a small number of frames"""
for i in range(200):

# to process whole video    
#while(True):
    
    # Capture frame-by-frame
    ret, im = cap.read()
    if not ret: break

    '''
    # For saving still images
    name = 'pframe_pi45_' + str(currentFrame) + '.jpg'
    save_path = os.path.join(out_path, name)
    print ('Creating...' + name)
    '''
    
    if crop_frame_border:
        im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    
    # Do processing
    img, _ = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
    
    """Save still image in jpeg format
    cv2.imwrite(save_path, img)"""
    
    """For annotating video
    @param org Bottom-left corner of the text string (default=50).
    @param org Bottom-left corner of the text string in the image (default=50).
    @param fontFace Font type, see #HersheyFonts (default=cv2.FONT_HERSHEY_PLAIN).
    @param fontScale Font scale factor that is multiplied 
                     by the font-specific base size (default=2).
    @param color Text color (default=(R=0, G=255, B=0)).
    @param thickness Thickness of the lines used to draw a text (default=1).
    @param lineType Line type. See #LineTypes (default=cv2.LINE_AA)."""
    cv2.putText(                 # x, y
        img, str(currentFrame), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 1, cv2.LINE_AA)
    vout.write(img)    

    # increment frame counter
    currentFrame += 1

"""When everything done, release the 
capture to flush the video stream. 

Before re-running this cell, first do 
cells 5, 6, and 7 to re-open cap and 
vout, otherwise ret=False and this 
cell does nothing."""
cap.release()
vout.release()

Found 4 lines
2 [[1114, 21], [1200, 636]]
Found 3 lines
1 [[1112, 23], [1208, 703]]
Found 5 lines
2 [[1115, 24], [1211, 704]]
Found 4 lines
2 [[1115, 24], [1211, 704]]
Found 6 lines
5 [[591, 54], [502, 686]]
3 [[1114, 23], [1210, 704]]
Found 4 lines
Found 6 lines
3 [[588, 50], [499, 683]]
Found 7 lines
4 [[581, 47], [492, 680]]
2 [[582, 47], [493, 683]]
Found 5 lines
Found 8 lines
5 [[411, 671], [1202, 699]]
3 [[577, 41], [487, 681]]
Found 9 lines
8 [[408, 668], [1197, 695]]
6 [[574, 39], [485, 674]]
3 [[510, 86], [1184, 63]]
Found 7 lines
4 [[572, 36], [482, 671]]
Found 10 lines
9 [[417, 585], [1257, 615]]
5 [[569, 33], [478, 677]]
4 [[571, 33], [480, 677]]
3 [[506, 78], [1170, 55]]
Found 10 lines
9 [[400, 661], [1214, 689]]
5 [[502, 81], [1162, 58]]
4 [[566, 32], [476, 672]]
3 [[568, 32], [478, 672]]
Found 7 lines
6 [[397, 662], [1173, 689]]
4 [[563, 30], [473, 672]]
Found 7 lines
6 [[560, 29], [471, 665]]
5 [[496, 78], [1157, 55]]
3 [[496, 75], [1162, 52]]
2 [[562, 28], [473, 665]]


8 [[383, 66], [1084, 41]]
6 [[1007, 1], [1058, 722]]
5 [[445, 12], [396, 713]]
4 [[446, 12], [397, 713]]
3 [[1006, 0], [1056, 722]]
Found 12 lines
11 [[1006, 1], [1056, 722]]
10 [[381, 65], [1082, 40]]
9 [[443, 12], [394, 713]]
8 [[443, 12], [394, 713]]
6 [[1006, 0], [1056, 722]]
5 [[380, 66], [1081, 42]]
4 [[1004, 0], [1055, 722]]
3 [[444, 12], [395, 713]]
Found 8 lines
7 [[378, 63], [1079, 39]]
5 [[443, 11], [394, 713]]
4 [[444, 11], [395, 713]]
3 [[443, 11], [394, 713]]
Found 11 lines
8 [[1002, 1], [1052, 720]]
7 [[1002, 0], [1052, 720]]
6 [[375, 65], [1063, 41]]
5 [[438, 10], [389, 713]]
4 [[440, 10], [391, 709]]
3 [[1001, 0], [1051, 720]]
Found 10 lines
8 [[999, 1], [1050, 720]]
7 [[1000, 1], [1051, 720]]
6 [[437, 9], [388, 713]]
5 [[373, 65], [1076, 40]]
4 [[436, 9], [387, 713]]
2 [[438, 9], [389, 713]]
Found 8 lines
5 [[1000, 0], [1050, 719]]
4 [[998, 0], [1048, 719]]
3 [[435, 9], [386, 713]]
Found 6 lines
5 [[999, 0], [1056, 808]]
3 [[438, 8], [382, 808]]
2 [[436, 8], [380, 808

Found 15 lines
14 [[390, 1], [338, 755]]
13 [[327, 40], [1097, 13]]
9 [[389, 1], [336, 754]]
7 [[262, 665], [1145, 695]]
6 [[327, 39], [1101, 11]]
4 [[261, 665], [1145, 696]]
Found 15 lines
14 [[1019, 0], [1070, 729]]
10 [[382, 49], [333, 758]]
9 [[325, 37], [1099, 10]]
7 [[260, 667], [1200, 700]]
6 [[324, 38], [1099, 11]]
3 [[260, 666], [1201, 699]]
Found 15 lines
14 [[384, 1], [335, 699]]
12 [[323, 36], [1104, 9]]
10 [[1020, 0], [1075, 783]]
7 [[372, 764], [1057, 788]]
5 [[258, 670], [1147, 701]]
4 [[258, 669], [1147, 700]]
Found 18 lines
17 [[321, 35], [1105, 7]]
16 [[296, 761], [1153, 790]]
15 [[383, 0], [330, 762]]
14 [[291, 762], [1126, 791]]
13 [[257, 670], [1139, 701]]
11 [[256, 671], [1144, 702]]
10 [[1020, 0], [1075, 791]]
6 [[321, 35], [1107, 7]]
4 [[256, 670], [1145, 701]]
2 [[281, 762], [1124, 792]]
Found 16 lines
15 [[384, 0], [334, 720]]
14 [[382, 0], [330, 742]]
13 [[1020, 0], [1071, 732]]
12 [[318, 37], [1100, 10]]
11 [[254, 673], [1150, 704]]
9 [[255, 672], [1150, 703

Found 15 lines
14 [[1084, 0], [1136, 750]]
13 [[349, 98], [305, 715]]
11 [[1083, 0], [1135, 749]]
7 [[224, 676], [1221, 710]]
5 [[223, 677], [1221, 711]]
4 [[223, 678], [1221, 712]]
3 [[351, 96], [305, 746]]
Found 11 lines
10 [[354, 86], [310, 706]]
8 [[225, 677], [1222, 712]]
7 [[226, 675], [1222, 710]]
6 [[351, 90], [305, 741]]
3 [[225, 675], [1222, 710]]
Found 14 lines
13 [[1085, 0], [1141, 801]]
11 [[351, 86], [304, 765]]
9 [[224, 674], [1222, 709]]
8 [[224, 673], [1222, 708]]
6 [[349, 88], [301, 769]]
5 [[223, 674], [1222, 709]]
Found 11 lines
10 [[320, 776], [1162, 805]]
6 [[219, 673], [1279, 710]]
4 [[219, 671], [1250, 707]]


You should see new mp4 files when listing current dir. You may not be able to view them directly in Google Drive,
but these can be downloaded to your local machine.

In [ ]:
%ls

# End Demo

What follows is additional code for debugging

In [ ]:
"""
# to seek into a specific frame
for _ in range(28):
    ret, im = cap.read()
"""    
"""
ret, im = cap.read()
if ret is not None:
    plt.imshow(im)
"""

In [ ]:
'''
if crop_frame_border:
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]

img, edges = draw_lines(im, rho=1, theta=np.pi/45, mll=mLL,
                        mlg=mLG, threshold=125, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
'''                        

In [ ]:
#plt.figure(figsize=(14, 12))
#plt.imshow(img)
#plt.imshow(im[y_trim:-y_trim, x_trim:-x_trim, :])

In [ ]:
#plt.imshow(edges)

In [ ]:
"""it can be helpful to save particularly difficult
frames for processing in a different notebook"""
#cv2.imwrite('test.jpg', im)

In [ ]:
"""
#cv2.circle(img, (800, 400), 10, (255, 0, 0), thickness=2, lineType=8, shift=0)
plt.figure(figsize=figsize)
plt.imshow(img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()
"""